In [ ]:
import numpy as np
import joblib

In [ ]:
class TrainInductionEnv:
    def __init__(self, max_trains=20):
        self.max_trains = max_trains

    def discretize_demand(self, demand):
        if demand < 2500:
            return 0   # Low
        elif demand < 4500:
            return 1   # Medium
        elif demand < 6500:
            return 2   # High
        else:
            return 3   # Very High

    def discretize_wait(self, wait):
        if wait < 4:
            return 0
        elif wait < 8:
            return 1
        else:
            return 2

    def discretize_crowding(self, ratio):
        if ratio < 0.8:
            return 0
        elif ratio < 1.1:
            return 1
        else:
            return 2

    def get_state(self, demand, peak, available_trains):
        demand_level = self.discretize_demand(demand)

        # Initial assumptions
        avg_wait = 6 if peak else 4
        crowding_ratio = demand / max(1, available_trains * 400)

        wait_level = self.discretize_wait(avg_wait)
        crowd_level = self.discretize_crowding(crowding_ratio)

        return (
            demand_level,
            peak,
            min(available_trains, self.max_trains),
            wait_level,
            crowd_level
        )

    def step(self, state, action, demand):
        demand_level, peak, available_trains, _, _ = state
        deployed_trains = action

        # ❌ Invalid action
        if deployed_trains < 2 or deployed_trains > available_trains:
            return -200

        # 🚦 System dynamics
        headway = max(2, 14 - deployed_trains)
        waiting_time = headway / 2
        capacity = deployed_trains * 400
        crowding_ratio = demand / capacity

        # ⚡ Costs
        energy_cost = deployed_trains * 1.5
        wait_penalty = waiting_time * 12

        overcrowding_penalty = 0
        if crowding_ratio > 1.3:
            overcrowding_penalty = 120
        elif crowding_ratio > 1.1:
            overcrowding_penalty = 60

        # 🎯 Passenger satisfaction bonus
        satisfaction = 0
        if waiting_time < 5 and crowding_ratio < 1.0:
            satisfaction = 40

        # 🧠 Final reward (BALANCED)
        reward = (
            -energy_cost
            -wait_penalty
            -overcrowding_penalty
            + satisfaction
        )

        return reward

In [ ]:
class QLearningAgent:
    def __init__(self, actions, alpha=0.15, gamma=0.95,
                 epsilon=1.0, epsilon_min=0.05, epsilon_decay=0.999):
        self.q_table = {}
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

    def get_q(self, state, action):
        return self.q_table.get((state, action), 0.0)

    def choose_action(self, state):
        # ε-greedy with slow decay
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.actions)

        qs = [self.get_q(state, a) for a in self.actions]
        return self.actions[np.argmax(qs)]

    def update(self, state, action, reward):
        best_future = max(self.get_q(state, a) for a in self.actions)
        old_q = self.get_q(state, action)

        self.q_table[(state, action)] = old_q + self.alpha * (
            reward + self.gamma * best_future - old_q
        )

    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
env = TrainInductionEnv(max_trains=20)
actions = list(range(2, 21))
agent = QLearningAgent(actions)

EPISODES = 6000

for episode in range(EPISODES):
    demand = np.random.randint(1200, 8000)
    peak = np.random.choice([0, 1])
    available = np.random.randint(6, 20)

    state = env.get_state(demand, peak, available)

    action = agent.choose_action(state)
    reward = env.step(state, action, demand)

    agent.update(state, action, reward)
    agent.decay_epsilon()

    if episode % 500 == 0:
        print(
            f"Episode {episode} | ε={agent.epsilon:.3f} | "
            f"demand={demand} peak={peak} available={available} "
            f"action={action} reward={reward:.1f}"
        )

In [ ]:
joblib.dump(agent.q_table, "../model/rl_q_table.pkl")
print("✅ RL model trained and saved.")

['../model/rl_q_table.pkl']